# MCS-GA-One-and-Two-Words model for long text summarization

## Start of [1]: Import data

In [1]:
import pandas as pd

Show training data head

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)
val = pd.read_csv("Dataset/cnn_dailymail/validation.csv")
test = pd.read_csv("Dataset/cnn_dailymail/test.csv")
val.head()

,id,article,highlights
0,61df4979ac5fcc2b71be46ed6fe5a46ce7f071c3,"Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films such as the 1956 noir While the City Sleeps died on March 15 at her home in Beverly Hills, California. Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer. Her publicist, Judith Goffin, announced the news Thursday. Scroll down for video . Actress: Sally Forrest was in the 1951 Ida Lupino-directed film 'Hard, Fast and Beautiful' (left) and the 1956 Fritz Lang movie 'While the City Sleeps' A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films including the critical and commercial success Not Wanted, Never Fear and Hard, Fast and Beautiful. Some of Forrest's other film credits included Bannerline, Son of Sinbad, and Excuse My Dust, according to her iMDB page. The page also indicates Forrest was in multiple Climax! and Rawhide television episodes. Forrest appeared as herself in an episode of The Ed Sullivan Show and three episodes of The Dinah Shore Chevy Show, her iMDB page says. She also starred in a Broadway production of The Seven Year Itch. City News Service reported that other stage credits included As You Like It, No, No, Nanette and Damn Yankees. Forrest married writer-producer Milo Frank in 1951. He died in 2004. She is survived by her niece, Sharon Durham, and nephews, Michael and Mark Feeney. Career: A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films .","Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films died on March 15 .\nForrest, whose birth name was Katherine Feeney, had long battled cancer .\nA San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films ."
1,21c0bd69b7e7df285c3d1b1cf56d4da925980a68,"A middle-school teacher in China has inked hundreds of sketches that are beyond be-leaf. Politics teacher Wang Lian, 35, has created 1000 stunning ink drawings covering subjects as varied as cartoon characters and landscapes to animals, birds according to the People's Daily Online. The intricate scribbles on leaves feature Wang's favourite sites across the city of Nanjing, which include the Presidential Palace, Yangtze River Bridge, the ancient Jiming Temple and the Qinhuai River. Natural canvas: Artist and teacher Wang Lian has done hundreds of drawings, like this temple, on leaves she collects in the park and on the streets . Delicate: She uses an ink pen to gently draw the local scenes and buildings on the dried out leaves . 'Although teaching politics is my job, drawing is my passion and hobby,' said Wang. 'I first tried drawing on leaves about 10 years ago and fell in love with it as an art form immediately. 'It's like drawing on very old parchment paper, you have to be really careful that you don't damage the leaf because it is very fragile and this helps focus your attention and abilities.' Wang started giving the drawings away on Christmas Eve in 2012 when her junior high school son came home saying he wanted to prepare some gifts for his classmates. Being an avid painter, Wang decided to give her son's friends unique presents of gingko leaf paintings. Wang loves gingko leaves and will often pick them up along Gingko Avenue, near to her school, in Nanjing in east China's Jiangsu province. Every autumn she collects about 2,000 leaves from the ground to ensure she has enough to cover spoils too. Intricate: Teacher Wang has drawn hundreds of local scenes on leaves she has collected from the park . Hobby: The artist collects leaves every autumn and dries them out so she can sketch these impressive building scenes . 'The colour and shape of gingko leaves are particularly beautiful,' she said. 'I need to collect around 2000 leaves because this wil

In [3]:
print(val.iloc[0]['article'])

Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films such as the 1956 noir While the City Sleeps died on March 15 at her home in Beverly Hills, California. Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer. Her publicist, Judith Goffin, announced the news Thursday. Scroll down for video . Actress: Sally Forrest was in the 1951 Ida Lupino-directed film 'Hard, Fast and Beautiful' (left) and the 1956 Fritz Lang movie 'While the City Sleeps' A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films including the critical and commercial success Not Wanted, Never Fear and Hard, Fast and Beautiful. Some of Forrest's other film credits included Bannerline, Son of Sinbad, and Excuse My Dust, according to her iMDB page. The page also indicates Forrest was in multiple Climax! and Rawhide television episodes. Forrest appeared as herself in

## End of [1]: Import data

## Start of [2]: Span recognition (sentence splitting in this model)

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/toilanvd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from nltk.tokenize import sent_tokenize, word_tokenize
sent_tokenize("Hello! Hello world... It's true that Mr.Ryo is here? That's true!")

['Hello!', 'Hello world...', "It's true that Mr.Ryo is here?", "That's true!"]

In [6]:
val_chapters = val['article'].tolist()
val_chapters_paragraphs = [val_chapters[i].split("\n\n") for i in range(len(val_chapters))]
for i in range(len(val_chapters_paragraphs)):
    val_chapters_paragraphs[i] = [val_chapters_paragraphs[i][j].replace("\n", " ") for j in range(len(val_chapters_paragraphs[i]))]
val_chapters = [val_chapters[i].replace("\n", " ") for i in range(len(val_chapters))]
test_chapters = test['article'].tolist()
test_chapters_paragraphs = [test_chapters[i].split("\n\n") for i in range(len(test_chapters))]
for i in range(len(test_chapters_paragraphs)):
    test_chapters_paragraphs[i] = [test_chapters_paragraphs[i][j].replace("\n", " ") for j in range(len(test_chapters_paragraphs[i]))]
test_chapters = [test_chapters[i].replace("\n", " ") for i in range(len(test_chapters))]

print(val_chapters_paragraphs[0])

["Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films such as the 1956 noir While the City Sleeps died on March 15 at her home in Beverly Hills, California. Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer. Her publicist, Judith Goffin, announced the news Thursday. Scroll down for video . Actress: Sally Forrest was in the 1951 Ida Lupino-directed film 'Hard, Fast and Beautiful' (left) and the 1956 Fritz Lang movie 'While the City Sleeps' A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films including the critical and commercial success Not Wanted, Never Fear and Hard, Fast and Beautiful. Some of Forrest's other film credits included Bannerline, Son of Sinbad, and Excuse My Dust, according to her iMDB\xa0page. The page also indicates Forrest was in multiple Climax! and Rawhide television episodes. Forrest appeared as herse

In [7]:
val_chapters_sentences = [sent_tokenize(val_chapters[i]) for i in range(len(val_chapters))]
test_chapters_sentences = [sent_tokenize(test_chapters[i]) for i in range(len(test_chapters))]

val_chapters_paragraphs_sentences = []
for i in range(len(val_chapters_paragraphs)): # chapter i
    chapter_list = []
    for j in range(len(val_chapters_paragraphs[i])): # paragraph j
        chapter_list.append(sent_tokenize(val_chapters_paragraphs[i][j]))
    val_chapters_paragraphs_sentences.append(chapter_list)
test_chapters_paragraphs_sentences = []
for i in range(len(test_chapters_paragraphs)): # chapter i
    chapter_list = []
    for j in range(len(test_chapters_paragraphs[i])): # paragraph j
        chapter_list.append(sent_tokenize(test_chapters_paragraphs[i][j]))
    test_chapters_paragraphs_sentences.append(chapter_list)

print(val_chapters_paragraphs_sentences[0])

[["Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films such as the 1956 noir While the City Sleeps died on March 15 at her home in Beverly Hills, California.", 'Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer.', 'Her publicist, Judith Goffin, announced the news Thursday.', 'Scroll down for video .', "Actress: Sally Forrest was in the 1951 Ida Lupino-directed film 'Hard, Fast and Beautiful' (left) and the 1956 Fritz Lang movie 'While the City Sleeps' A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films including the critical and commercial success Not Wanted, Never Fear and Hard, Fast and Beautiful.", "Some of Forrest's other film credits included Bannerline, Son of Sinbad, and Excuse My Dust, according to her iMDB\xa0page.", 'The page also indicates Forrest was in multiple Climax!', 'and Rawhide television episodes.', '

## End of [2]: Span recognition

## Start of [3]: (BERT + cosine) score function for sentence similarity

In [8]:
import torch
from sentence_transformers import SentenceTransformer, util

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
sentence_embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device = device)  # multi-language model  

def sentence_similarity(sentence_A, sentence_B):
    temp_sentence_A = [sentence_A]
    temp_embedding_A = sentence_embedding_model.encode(temp_sentence_A, convert_to_tensor=False)
    temp_sentence_B = [sentence_B]
    temp_embedding_B = sentence_embedding_model.encode(temp_sentence_B, convert_to_tensor=False)
    cosine_score = util.cos_sim(temp_embedding_A, temp_embedding_B)
    return cosine_score[0][0].item()

print(val_chapters_sentences[0][2])
print(val_chapters_sentences[0][3])
print(sentence_similarity(val_chapters_sentences[0][2], val_chapters_sentences[0][3]))

/home/toilanvd/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu
Her publicist, Judith Goffin, announced the news Thursday.
Scroll down for video .
0.03955427557229996


## End of [3]: (BERT + cosine) score function for sentence similarity

## Start of [5']: Metrics preparation

In [9]:
# Validation reference summaries
val_reference_summaries = val['highlights'].tolist()
val_reference_summaries = [val_reference_summaries[i].replace("\n", " ") for i in range(len(val_reference_summaries))]

In [10]:
# Calculate average summary length vs chapter length ratio
average_summary_length_ratio = 0
for i in range(len(val_chapters)):
    average_summary_length_ratio += len(word_tokenize(val_reference_summaries[i])) / len(word_tokenize(val_chapters[i]))
average_summary_length_ratio /= len(val_chapters)
print(average_summary_length_ratio)

0.09802262149041631


In [11]:
import copy
import math
import numpy as np
import os
from sklearn.preprocessing import normalize

EPS = 0.000001

In [12]:
from rouge_score import rouge_scorer
rougescorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [13]:
sent_A = "1 I love machine learning"
sent_B = "1 think love i machine learning.."
print(rougescorer.score(sent_B, sent_A))

{'rouge1': Score(precision=1.0, recall=0.8333333333333334, fmeasure=0.9090909090909091), 'rouge2': Score(precision=0.25, recall=0.2, fmeasure=0.22222222222222224), 'rougeL': Score(precision=0.8, recall=0.6666666666666666, fmeasure=0.7272727272727272)}


In [14]:
import bert_score
from bert_score import BERTScorer

bertscorer = BERTScorer(lang='en', rescale_with_baseline=True) # Default using 'roberta-large' model
#bertscorer = BERTScorer(model_type='xlnet-large-cased', rescale_with_baseline=True, lang="en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from summaqa import evaluate_corpus
from summaqa import QG_masked
question_generator = QG_masked()
from summaqa import QA_Metric
qa_metric = QA_Metric()

/home/toilanvd/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## End of [5']: Metrics preparation

## Start of [6']: Run test & evaluate

In [16]:
# Test reference summaries
test_reference_summaries = test['highlights'].tolist()
test_reference_summaries = [test_reference_summaries[i].replace("\n", " ") for i in range(len(test_reference_summaries))]

In [20]:
# Test summary generation
test_generated_summaries = []
for i in range(len(test_chapters)):
    num_sentences = len(test_chapters_sentences[i])
    print("Test doc {}: {} sentences".format(i, len(test_chapters_sentences[i])))
    summary_length_limit = math.ceil(len(word_tokenize(test_chapters[i])) * average_summary_length_ratio)
    
    temp_sentences_embedding = sentence_embedding_model.encode(test_chapters_sentences[i], convert_to_tensor=False)
    cosine_scores = util.cos_sim(temp_sentences_embedding, temp_sentences_embedding)
    
    sentence_sum_similarity = [(0, j) for j in range(len(test_chapters_sentences[i]))]
    for j in range(len(test_chapters_sentences[i])):
        for k in range(len(test_chapters_sentences[i])):
            #print("j = {} - k = {}".format(j, k))
            sentence_sum_similarity[j] = (sentence_sum_similarity[j][0] + cosine_scores[j][k], j)
    sentence_sum_similarity.sort(reverse = True)
    generated_summary_length = 0
    chosen_sentences_index = []
    for j in range(len(test_chapters_sentences[i])):
        chosen_sentences_index.append(sentence_sum_similarity[j][1])
        generated_summary_length += len(' '.join(test_chapters_sentences[i][sentence_sum_similarity[j][1]].split(" ")).split())
        if generated_summary_length >= summary_length_limit:
            break
    chosen_sentences_index.sort()
    generated_summary = ""
    for j in range(len(chosen_sentences_index)):
        if j > 0:
            generated_summary += " "
        generated_summary += test_chapters_sentences[i][chosen_sentences_index[j]]
    test_generated_summaries.append(generated_summary)

Test doc 0: 16 sentences
Test doc 1: 14 sentences
Test doc 2: 6 sentences
Test doc 3: 15 sentences
Test doc 4: 34 sentences
Test doc 5: 17 sentences
Test doc 6: 20 sentences
Test doc 7: 19 sentences
Test doc 8: 47 sentences
Test doc 9: 45 sentences
Test doc 10: 47 sentences
Test doc 11: 28 sentences
Test doc 12: 7 sentences
Test doc 13: 57 sentences
Test doc 14: 26 sentences
Test doc 15: 26 sentences
Test doc 16: 17 sentences
Test doc 17: 43 sentences
Test doc 18: 16 sentences
Test doc 19: 42 sentences
Test doc 20: 34 sentences
Test doc 21: 48 sentences
Test doc 22: 67 sentences
Test doc 23: 37 sentences
Test doc 24: 20 sentences
Test doc 25: 17 sentences
Test doc 26: 15 sentences
Test doc 27: 58 sentences
Test doc 28: 50 sentences
Test doc 29: 28 sentences
Test doc 30: 36 sentences
Test doc 31: 20 sentences
Test doc 32: 30 sentences
Test doc 33: 31 sentences
Test doc 34: 24 sentences
Test doc 35: 17 sentences
Test doc 36: 31 sentences
Test doc 37: 21 sentences
Test doc 38: 33 sentence

In [18]:
print(test_generated_summaries[0])

They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. But these tests are conducted using planes with 31 inches between each row of seats, a standard which on some airlines has decreased, reported the Detroit News.


In [19]:
# Test ROUGE score
test_rouge1 = 0
test_rouge2 = 0
test_rougeL = 0
for i in range(len(test_generated_summaries)):
    print(i)
    scores = rougescorer.score(test_reference_summaries[i], test_generated_summaries[i])
    for key in scores:
        print("{}: {}".format(key, scores[key]))
        if key == "rouge1":
            test_rouge1 += scores[key][2] # take fmeasure value
        elif key == "rouge2":
            test_rouge2 += scores[key][2] # take fmeasure value
        else:
            test_rougeL += scores[key][2] # take fmeasure value
test_rouge1 /= len(test_generated_summaries)
test_rouge2 /= len(test_generated_summaries)
test_rougeL /= len(test_generated_summaries)
print("Test: rouge1 = {}, rouge2 = {}, rougeL = {}".format(test_rouge1, test_rouge2, test_rougeL))

0
rouge1: Score(precision=0.25, recall=0.35294117647058826, fmeasure=0.2926829268292683)
rouge2: Score(precision=0.02127659574468085, recall=0.030303030303030304, fmeasure=0.024999999999999998)
rougeL: Score(precision=0.16666666666666666, recall=0.23529411764705882, fmeasure=0.19512195121951217)
1
rouge1: Score(precision=0.44285714285714284, recall=0.8611111111111112, fmeasure=0.5849056603773586)
rouge2: Score(precision=0.2608695652173913, recall=0.5142857142857142, fmeasure=0.34615384615384615)
rougeL: Score(precision=0.34285714285714286, recall=0.6666666666666666, fmeasure=0.4528301886792453)
2
rouge1: Score(precision=0.42857142857142855, recall=0.36363636363636365, fmeasure=0.3934426229508197)
rouge2: Score(precision=0.1111111111111111, recall=0.09375, fmeasure=0.10169491525423728)
rougeL: Score(precision=0.25, recall=0.21212121212121213, fmeasure=0.22950819672131145)
3
rouge1: Score(precision=0.37209302325581395, recall=0.4, fmeasure=0.38554216867469876)
rouge2: Score(precision=0.1

In [ ]:
# Print test result to file
test_result_file = open("MCS-GA-One-and-Two-Words_CNN-Dailymail_test-generated-result.txt", "w")
for i in range(len(test_generated_summaries)):
    print(test_generated_summaries[i], file = test_result_file)
test_result_file.close()

In [ ]:
# Test BERT score
average_bertscore_F1 = 0

for i in range(len(test_generated_summaries)):
    print("Test doc {}:".format(i))
    
    ref_summary = copy.deepcopy(test_reference_summaries[i])
    candidate_summary = copy.deepcopy(test_generated_summaries[i])

    # Method 1: Split by sentence
    ref_sentences = sent_tokenize(ref_summary)
    candidate_sentences = sent_tokenize(candidate_summary)
    chapter_bertscore_F1 = 0
    for j in range(len(ref_sentences)):
        ref_summary_split = ref_sentences[j]
        max_candidate_score = -1
        for k in range(len(candidate_sentences)):
            candidate_summary_split = candidate_sentences[k]
            P, R, F1 = bertscorer.score([candidate_summary_split], [ref_summary_split])
            #print("Candidate length = {}, reference length = {}".format(len(word_tokenize(candidate_summary_split)), len(word_tokenize(ref_summary_split))))
            max_candidate_score = max(max_candidate_score, F1.mean().item())
        chapter_bertscore_F1 += max_candidate_score
    chapter_bertscore_F1 /= len(ref_sentences)
    average_bertscore_F1 += chapter_bertscore_F1
    print("Chapter BERTScore F1 = {}".format(chapter_bertscore_F1))

    '''
    # Method 2: Split by n words
    ref_tokenized = word_tokenize(ref_summary)
    candidate_tokenized = word_tokenize(candidate_summary)
    split_length = 250
    ref_pointer = 0
    ref_split_count = 0
    chapter_bertscore_F1 = 0
    while ref_pointer < len(ref_tokenized):
        ref_summary_split = ' '.join(ref_tokenized[ref_pointer:min(len(ref_tokenized), ref_pointer + split_length)])
        max_candidate_score = -1
        candidate_pointer = 0
        while candidate_pointer < len(candidate_tokenized):
            candidate_summary_split = ' '.join(candidate_tokenized[candidate_pointer:min(len(candidate_tokenized), candidate_pointer + split_length)])
            P, R, F1 = bertscorer.score([candidate_summary_split], [ref_summary_split])
            #print("Candidate length = {}, reference length = {}".format(len(word_tokenize(candidate_summary_split)), len(word_tokenize(ref_summary_split))))
            max_candidate_score = max(max_candidate_score, F1.mean().item())
            candidate_pointer += split_length
        chapter_bertscore_F1 += max_candidate_score
        ref_pointer += split_length
        ref_split_count += 1
    chapter_bertscore_F1 /= ref_split_count
    average_bertscore_F1 += chapter_bertscore_F1
    print("Chapter BERTScore F1 = {}".format(chapter_bertscore_F1))
    '''
average_bertscore_F1 /= len(test_generated_summaries)
print("Test average BERTscore F1 = {}".format(average_bertscore_F1))

In [ ]:
# Test summaQA
#srcs = test_chapters[:len(test_generated_summaries)]
#gens = test_generated_summaries
#srcs = [' '.join(' '.join(srcs[i].split(" ")).split()[:300]) for i in range(len(srcs))]
#gens = [' '.join(' '.join(gens[i].split(" ")).split()[:300]) for i in range(len(gens))]
#evaluate_corpus(srcs, gens)

average_summaqa_prob = 0
average_summaqa_F1 = 0

for i in range(len(test_generated_summaries)):
    print("Test doc {}:".format(i))
    
    chapter_content = copy.deepcopy(test_chapters[i])
    candidate_summary = copy.deepcopy(test_generated_summaries[i])

    # Method 1: Split by sentence
    chapter_sentences = test_chapters_sentences[i]
    candidate_sentences = sent_tokenize(candidate_summary)
    chapter_split_count = 0
    chapter_summaqa_prob = 0
    chapter_summaqa_F1 = 0
    for j in range(len(chapter_sentences)):
        chapter_summary_split = chapter_sentences[j]
        article = chapter_summary_split
        masked_questions, answer_spans = question_generator.get_questions(article)
        if len(masked_questions) == 0:
            continue
        chapter_split_count += 1
        print("Chapter split #{}:".format(j))
        print(chapter_summary_split)
        print("Questions:")
        print(masked_questions)
        print("Answers:")
        print(answer_spans)
        max_candidate_F1 = 0
        max_candidate_prob = 0
        #chapter_split_sum_F1 = 0
        #chapter_split_sum_prob = 0
        for k in range(len(candidate_sentences)):
            candidate_summary_split = candidate_sentences[k]
            summaqa_scores = qa_metric.compute(masked_questions, answer_spans, candidate_summary_split)
            max_candidate_F1 = max(max_candidate_F1, summaqa_scores['avg_fscore'])
            max_candidate_prob = max(max_candidate_prob, summaqa_scores['avg_prob'])
            #chapter_split_sum_F1 += summaqa_scores['avg_fscore']
            #chapter_split_sum_prob += summaqa_scores['avg_prob']
            print("Candidate split #{} (Chapter split length = {}, candidate length = {}): SummaQA score = {}".format(k, len(word_tokenize(chapter_summary_split)), len(word_tokenize(candidate_summary_split)), summaqa_scores))
            print(candidate_summary_split)
        print("Chapter split max prob = {}".format(max_candidate_prob))
        print("Chapter split max F1 = {}".format(max_candidate_F1))
        #print("Chapter split sum prob = {}".format(chapter_split_sum_prob))
        #print("Chapter split sum F1 = {}".format(chapter_split_sum_F1))
        chapter_summaqa_prob += max_candidate_prob
        chapter_summaqa_F1 += max_candidate_F1
        #chapter_summaqa_prob += chapter_split_sum_prob
        #chapter_summaqa_F1 += chapter_split_sum_F1
    if chapter_split_count > 0:
        chapter_summaqa_prob /= chapter_split_count
        chapter_summaqa_F1 /= chapter_split_count
    else:
        chapter_summaqa_prob = 1
        chapter_summaqa_F1 = 1
    average_summaqa_prob += chapter_summaqa_prob
    average_summaqa_F1 += chapter_summaqa_F1
    print("Chapter SummaQA prob = {}".format(chapter_summaqa_prob))
    print("Chapter SummaQA F1 = {}".format(chapter_summaqa_F1))
    
    '''
    # Method 2: Split by n words
    chapter_tokenized = word_tokenize(chapter_content)
    candidate_tokenized = word_tokenize(candidate_summary)
    split_length = 250
    chapter_pointer = 0
    chapter_split_count = 0
    chapter_summaqa_prob = 0
    chapter_summaqa_F1 = 0
    while chapter_pointer < len(chapter_tokenized):
        chapter_summary_split = ' '.join(chapter_tokenized[chapter_pointer:min(len(chapter_tokenized), chapter_pointer + split_length)])
        article = chapter_summary_split
        masked_questions, answer_spans = question_generator.get_questions(article)
        print("Chapter split #{}:".format(chapter_split_count))
        print(chapter_summary_split)
        print("Questions:")
        print(masked_questions)
        print("Answers:")
        print(answer_spans)
        #max_candidate_score = -1
        chapter_split_sum_F1 = 0
        chapter_split_sum_prob = 0
        candidate_pointer = 0
        while candidate_pointer < len(candidate_tokenized):
            candidate_summary_split = ' '.join(candidate_tokenized[candidate_pointer:min(len(candidate_tokenized), candidate_pointer + split_length)])
            summaqa_scores = qa_metric.compute(masked_questions, answer_spans, candidate_summary_split)
            #max_candidate_score = max(max_candidate_score, summaqa_scores['avg_fscore'])
            chapter_split_sum_F1 += summaqa_scores['avg_fscore']
            chapter_split_sum_prob += summaqa_scores['avg_prob']
            print("Candidate split #{} (Chapter split length = {}, candidate length = {}): SummaQA score = {}".format(math.floor(candidate_pointer / split_length), len(word_tokenize(chapter_summary_split)), len(word_tokenize(candidate_summary_split)), summaqa_scores))
            print(candidate_summary_split)
            candidate_pointer += split_length
        print("Chapter split sum prob = {}".format(chapter_split_sum_prob))
        print("Chapter split sum F1 = {}".format(chapter_split_sum_F1))
        chapter_summaqa_prob += chapter_split_sum_prob
        chapter_summaqa_F1 += chapter_split_sum_F1
        #chapter_summaqa_F1 += max_candidate_score
        chapter_pointer += split_length
        chapter_split_count += 1
    chapter_summaqa_prob /= chapter_split_count
    chapter_summaqa_F1 /= chapter_split_count
    average_summaqa_prob += chapter_summaqa_prob
    average_summaqa_F1 += chapter_summaqa_F1
    print("Chapter SummaQA prob = {}".format(chapter_summaqa_prob))
    print("Chapter SummaQA F1 = {}".format(chapter_summaqa_F1))
    '''

average_summaqa_prob /= len(test_generated_summaries)
average_summaqa_F1 /= len(test_generated_summaries)
print("Test average SummaQA prob = {}".format(average_summaqa_prob))
print("Test average SummaQA F1 = {}".format(average_summaqa_F1))

## End of [6']: Run test & evaluate